In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
df=pd.read_csv("insurance.csv")
df.head()


,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [ ]:
y

0     0
1     0
2     1
3     0
4     1
5     1
6     0
7     1
8     1
9     1
10    0
11    0
12    0
13    0
14    1
15    1
16    1
17    1
18    0
19    0
20    0
21    0
22    1
23    1
24    1
25    1
26    0
27    0
Name: bought_insurance, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)


In [ ]:
X_train_scaled=X_train.copy()
X_train_scaled['age']=X_train_scaled['age']/100
X_test_scaled=X_test.copy()
X_test_scaled['age']=X_test_scaled['age']/100


In [ ]:
X_test_scaled.head()

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1


In [ ]:
model=keras.Sequential([
                        keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',
                        kernel_initializer='one',bias_initializer='zeros')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')
model.fit(X_train_scaled,y_train,epochs=6000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 13ms/step - loss: 0.5119 - accuracy: 0.8182
Epoch 3502/6000
1/1 [==============================] - 0s 15ms/step - loss: 0.5119 - accuracy: 0.8182
Epoch 3503/6000
1/1 [==============================] - 0s 12ms/step - loss: 0.5119 - accuracy: 0.8182
Epoch 3504/6000
1/1 [==============================] - 0s 14ms/step - loss: 0.5118 - accuracy: 0.8182
Epoch 3505/6000
1/1 [==============================] - 0s 12ms/step - loss: 0.5118 - accuracy: 0.8182
Epoch 3506/6000
1/1 [==============================] - 0s 13ms/step - loss: 0.5118 - accuracy: 0.8182
Epoch 3507/6000
1/1 [==============================] - 0s 16ms/step - loss: 0.5117 - accuracy: 0.8182
Epoch 3508/6000
1/1 [==============================] - 0s 13ms/step - loss: 0.5117 - accuracy: 0.8182
Epoch 3509/6000
1/1 [==============================] - 0s 12ms/step - loss: 0.5117 - accuracy: 0.8182
Epoch 3510/6000
1/1 [==========================

In [ ]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 27ms/step - loss: 0.2740 - accuracy: 1.0000


[0.27396056056022644, 1.0]

In [ ]:
model.predict(X_test_scaled)

array([[0.13526127],
       [0.37258807],
       [0.7413501 ],
       [0.8522817 ],
       [0.31987762],
       [0.8288794 ]], dtype=float32)

In [ ]:
y_test

1     0
0     0
14    1
9     1
19    0
17    1
Name: bought_insurance, dtype: int64

In [ ]:
coef, intercept = model.get_weights()
coef, intercept


(array([[5.8301024],
        [1.5089923]], dtype=float32), array([-3.3127449], dtype=float32))

In [ ]:
def sigmoid(x):
  import math
  return 1/(1+math.exp(-x))
sigmoid(5)

0.9933071490757153

In [ ]:
def prediction_function(age,affordibility):
  weight_sum=coef[0]*age+coef[1]*affordibility+intercept
  return sigmoid(weight_sum)

In [ ]:
prediction_function(.25,0)

0.13526126165800964

In [ ]:
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [ ]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [ ]:
class NN:
    def __init__(self):
        self.w1 = 1 
        self.w2 = 1
        self.bias = 0
        
    def fit(self, X, y, epochs, loss_thresold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'],X['affordibility'],y, epochs, loss_thresold)
        print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, bias: {self.bias}")
        
    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
        return sigmoid_numpy(weighted_sum)

    def gradient_descent(self, age,affordability, y_true, epochs, loss_thresold):
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)
            
            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d
            
            if i%50==0:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
            
            if loss<=loss_thresold:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1, w2, bias

In [ ]:
customModel=NN()
customModel.fit(X_train_scaled,y_train,epochs=500,loss_thresold=0.4631)


Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
Epoch:50, w1:1.503319554173139, w2:1.108384790367645, bias:-1.2319047301235464, loss:0.5675865113475955
Epoch:100, w1:2.200713131760032, w2:1.2941584023238903, bias:-1.6607009122062801, loss:0.5390680417774752
Epoch:150, w1:2.8495727769689085, w2:1.3696895491572745, bias:-1.986105845859897, loss:0.5176462164249293
Epoch:200, w1:3.443016970881803, w2:1.4042218624465033, bias:-2.2571369883752723, loss:0.5005011269691375
Epoch:250, w1:3.982450494649576, w2:1.4239127329321233, bias:-2.494377365971801, loss:0.48654089537617085
Epoch:300, w1:4.472179522095915, w2:1.438787986553552, bias:-2.707387811922373, loss:0.4750814640632793
Epoch:350, w1:4.917245868007634, w2:1.4525660781176122, bias:-2.901176333556766, loss:0.46561475306999006
Epoch:366, w1:5.051047623653049, w2:1.4569794548473887, bias:-2.9596534546250037, loss:0.46293944095888906
Final weights and bias: w1: 5.051047623653049, w2: 